<a href="https://colab.research.google.com/github/Jihaan-Jain/Text-Classification/blob/main/word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import numpy as np
import pandas as pd

In [11]:
temp_df = pd.read_csv('IMDB Dataset.csv', engine='python', on_bad_lines='skip')


In [12]:
df = temp_df.iloc[:10000]


In [14]:
df.head()


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [15]:
df.drop_duplicates(inplace=True)


/tmp/ipython-input-15-3617111529.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


In [16]:
import re
def remove_tags(raw_text):
    cleaned_text = re.sub(re.compile('<.*?>'), '', raw_text)
    return cleaned_text

In [17]:
df['review'] = df['review'].apply(remove_tags)


/tmp/ipython-input-17-1164145055.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(remove_tags)


In [18]:
df['review'] = df['review'].apply(lambda x:x.lower())


/tmp/ipython-input-18-3523187300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x:x.lower())


In [19]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
from nltk.corpus import stopwords

sw_list = stopwords.words('english')

df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))

/tmp/ipython-input-20-2826946130.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['review'].apply(lambda x: [item for item in x.split() if item not in sw_list]).apply(lambda x:" ".join(x))


In [21]:
df['review']


,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [22]:
!pip install gensim
import gensim


In [23]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [31]:
import nltk
nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |  

True

In [32]:
from nltk.tokenize import sent_tokenize
from gensim.utils import simple_preprocess

# Download the required tokenizer
nltk.download('punkt')

story = []

for doc in df['review']:
    raw_sent = sent_tokenize(doc)
    for sent in raw_sent:
        story.append(simple_preprocess(sent))

print("Tokenization complete!")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Tokenization complete!


In [33]:
model = gensim.models.Word2Vec(
    window=10,
    min_count=2
)

In [34]:
model.build_vocab(story)


In [35]:
model.train(story, total_examples=model.corpus_count, epochs=model.epochs)


(5849636, 6186875)

In [36]:
len(model.wv.index_to_key)


31845

In [37]:
def document_vector(doc):
    # remove out-of-vocabulary words
    doc = [word for word in doc.split() if word in model.wv.index_to_key]
    return np.mean(model.wv[doc], axis=0)

In [38]:
document_vector(df['review'].values[0])


array([-0.18530667,  0.47199267,  0.18506978, -0.02678268,  0.02657759,
       -0.6590408 , -0.01613432,  0.72073376, -0.32406712, -0.06664086,
       -0.12312115, -0.5358607 , -0.09049907,  0.08068262,  0.20309328,
       -0.09320039,  0.08790181, -0.5022433 , -0.09213787, -0.63383967,
        0.25203606,  0.0758686 ,  0.15607703, -0.2732463 , -0.16440634,
       -0.13134713, -0.31616223, -0.31429225, -0.2625535 ,  0.04269196,
        0.42076612, -0.0583086 ,  0.06518369, -0.32005984, -0.3682184 ,
        0.32814834, -0.01994728, -0.3150232 , -0.12625891, -0.8022603 ,
        0.06616797, -0.32580823, -0.2070979 ,  0.03502064,  0.32339445,
       -0.16078284, -0.4145721 , -0.05028034,  0.29365194,  0.21383224,
        0.15637064, -0.32434157, -0.28344658, -0.22529994, -0.27757922,
        0.14258565,  0.40260583,  0.07784756, -0.25026083,  0.0661907 ,
       -0.23657943,  0.18792789, -0.16485757, -0.09038357, -0.28713074,
        0.40709344,  0.14714333,  0.27237535, -0.7151363 ,  0.37

In [39]:
from tqdm import tqdm


In [40]:
X = []
for doc in tqdm(df['review'].values):
    X.append(document_vector(doc))

100%|██████████| 9983/9983 [05:19<00:00, 31.27it/s]


In [41]:

X = np.array(X)


In [42]:
X[0]


array([-0.18530667,  0.47199267,  0.18506978, -0.02678268,  0.02657759,
       -0.6590408 , -0.01613432,  0.72073376, -0.32406712, -0.06664086,
       -0.12312115, -0.5358607 , -0.09049907,  0.08068262,  0.20309328,
       -0.09320039,  0.08790181, -0.5022433 , -0.09213787, -0.63383967,
        0.25203606,  0.0758686 ,  0.15607703, -0.2732463 , -0.16440634,
       -0.13134713, -0.31616223, -0.31429225, -0.2625535 ,  0.04269196,
        0.42076612, -0.0583086 ,  0.06518369, -0.32005984, -0.3682184 ,
        0.32814834, -0.01994728, -0.3150232 , -0.12625891, -0.8022603 ,
        0.06616797, -0.32580823, -0.2070979 ,  0.03502064,  0.32339445,
       -0.16078284, -0.4145721 , -0.05028034,  0.29365194,  0.21383224,
        0.15637064, -0.32434157, -0.28344658, -0.22529994, -0.27757922,
        0.14258565,  0.40260583,  0.07784756, -0.25026083,  0.0661907 ,
       -0.23657943,  0.18792789, -0.16485757, -0.09038357, -0.28713074,
        0.40709344,  0.14714333,  0.27237535, -0.7151363 ,  0.37

In [43]:

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

y = encoder.fit_transform(df['sentiment'])

In [45]:
y


array([1, 1, 1, ..., 0, 0, 1])

In [46]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [48]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7696544817225839